# GRID TRADING STRATEGY

### IMPORTS

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
from multiprocessing import Process
from datetime import timedelta
import datetime 
from datetime import date, datetime
import time
from decimal import Decimal
from tinkoff.invest.utils import decimal_to_quotation, quotation_to_decimal

from tinkoff.invest import (
    CandleInstrument,
    Client,
    MarketDataRequest,
    SubscribeCandlesRequest,
    SubscriptionAction,
    SubscriptionInterval,
    OrderDirection,
    OrderType,
    SandboxPayInRequest
)

from tinkoff.invest.sandbox.client import SandboxClient

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

read = open(r"C:\Users\Dmitrii\Desktop\finance tests\token.txt", 'r')
for line in read:
    token = line
read.close()

TOKEN = token

### CLASS FOR BOT INFO

In [21]:
class info(object):
    def __init__(self,position_arr, profits, curent_date, firstlevels, secondlevels, firstbuy, secondbuy):
        self.position_arr = position_arr
        self.profits = profits
        self.curent_date = curent_date
        self.firstlevels = firstlevels
        self.secondlevels = secondlevels
        self.firstbuy = firstbuy
        self.secondbuy = secondbuy

In [22]:
def make_bot_info(ticker, acc_id):
    bot_info = 0
    try:
        bot_info_text_file = open(f'sandbox_bot_info_{ticker}_{acc_id}.txt','r')
        bot_info_text = bot_info_text_file.read()
        if len(bot_info_text) == 0:
            bot_info = info([False, False], 0,pd.to_datetime('1990-05-02'), [], [], 0, 0)
        else:
            rows = bot_info_text.split('\n')
            if rows[0].split(', ')[0][1:] == 'True':
                first = True
            else:
                first = False
            if rows[0].split(', ')[1][:-1] == 'True':
                second = True
            else:
                second = False
            position_array = [first, second]
            profits = float(rows[1])
            current_day = pd.to_datetime(rows[2])
            furstlevels = (float(rows[3].split(',')[0][1:]),float(rows[3].split(',')[1][:-1]))
            secondlevels = (float(rows[4].split(',')[0][1:]),float(rows[4].split(',')[1][:-1]))
            firstbuy = float(rows[5])
            secondbuy = float(rows[6])
            bot_info = info(position_array, profits,current_day,furstlevels,secondlevels,firstbuy, secondbuy)
        bot_info_text_file.close()
    except:
        bot_info = info([False, False], 0,pd.to_datetime('1990-05-02'),[],[], 0, 0)
    return bot_info

### FUNCTIONS FOR SANDBOX ASCCOUNT

In [23]:
def get_sandbox_account_id(name):
    with SandboxClient(TOKEN) as client:
        accounts = client.users.get_accounts().accounts
        for account in accounts:
            if account.name == name:
                return account.id
        return -1

In [24]:
def insert_money(acc_id, ammount):
    amount_dec = decimal_to_quotation(Decimal(str(ammount)))
    with SandboxClient(TOKEN) as client:
        balance = client.sandbox.sandbox_pay_in(account_id=acc_id, amount=amount_dec)
        return balance

In [25]:
def get_portfolio(acc_id):
    return client.sandbox.get_sandbox_portfolio(account_id=first_sandbox_acc_id)

### STRATEGY FUNCTIONS

In [26]:
def get_data_tink(ticker, date_needed):
    
    figi = get_figi(ticker)

    Y,m,d = int(date_needed.split('-')[0]), int(date_needed.split('-')[1]), int(date_needed.split('-')[2])
    dt_object = date(Y, m, d)
    delta = now().date() - dt_object
    
    frame = pd.DataFrame(columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    with Client(TOKEN) as client:
        for candle in client.get_all_candles(
            figi=figi,
            from_=now() - timedelta(days=delta.days),
            interval=CandleInterval.CANDLE_INTERVAL_1_MIN,
        ):
            time = pd.to_datetime(candle.time + timedelta(hours=3))
            open_ = candle.open.units+candle.open.nano*10**(-9)
            high_ = candle.high.units+candle.high.nano*10**(-9)
            low_ = candle.low.units+candle.low.nano*10**(-9)
            close_ = candle.close.units+candle.close.nano*10**(-9)
            volume_ = candle.volume
            new_row = [time, open_, high_, low_, close_, volume_]
            frame.loc[len(frame.index)] = new_row
    frame.set_index('Time', inplace = True)
    return frame

In [27]:
def get_figi(ticker):
    
    ticker = ticker
    
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        tickers = []
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                tickers.append(
                    {
                        "ticker": item.ticker,
                        "figi": item.figi,
                    }
                )

        tickers_df = pd.DataFrame(tickers)

        ticker_df = tickers_df[tickers_df["ticker"] == ticker]
        if ticker_df.empty:
            logger.error("There is no such ticker: %s", ticker)
            return

        figi = ticker_df["figi"].iloc[0]
        return figi

In [28]:
def get_lot(ticker):
    ticker = ticker
    
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        tickers = []
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                tickers.append(
                    {
                        "ticker": item.ticker,
                        "lot": item.lot,
                    }
                )

        tickers_df = pd.DataFrame(tickers)

        ticker_df = tickers_df[tickers_df["ticker"] == ticker]
        if ticker_df.empty:
            logger.error("There is no such ticker: %s", ticker)
            return

        lot = ticker_df["lot"].iloc[0]
        return lot

In [29]:
def buy(acc_id, ticker, price, quantity):
    lot = int(get_lot(ticker))
    lot_price = float(lot*price)
    figi = get_figi(ticker)
    with Client(TOKEN) as client:
        response = client.users.get_accounts()
        account, *_ = response.accounts
        response = client.sandbox.post_sandbox_order(
            figi=figi,
            quantity=quantity,
            price=decimal_to_quotation(Decimal(str(lot_price))),
            direction=OrderDirection.ORDER_DIRECTION_BUY,
            account_id = acc_id,
            order_type=OrderType.ORDER_TYPE_MARKET,
        )
    return response

In [30]:
def sell(acc_id, ticker, price, quantity):
    lot = int(get_lot(ticker))
    lot_price = float(lot*price)
    figi = get_figi(ticker)
    with Client(TOKEN) as client:
        response = client.users.get_accounts()
        account, *_ = response.accounts
        response = client.sandbox.post_sandbox_order(
            figi=figi,
            quantity=quantity,
            price=decimal_to_quotation(Decimal(str(lot_price))),
            direction=OrderDirection.ORDER_DIRECTION_SELL,
            account_id = acc_id,
            order_type=OrderType.ORDER_TYPE_MARKET,
        )
    return response

In [31]:
def get_first_open_price(date, ticker):
    df = get_data_tink(ticker, (date - timedelta(days=1)).strftime('%Y-%m-%d'))
    open_price = df.resample('1h').first().Open[date]
    return open_price

In [32]:
def getlevels(date, ticker, first=True):
    open_price = get_first_open_price(date, ticker)
    if first:
        return round(open_price * 0.998, 6), round(open_price * 1.002, 6)
    else:
        return round(open_price * 0.994, 6), round(open_price * 0.998, 6)

In [33]:
def new_strategy_logic(acc_id, ticker, low_, high_, bot_info, quantity):
    if not any(bot_info.position_arr) and bot_info.curent_date.strftime("%Y-%m-%d") != date.today().strftime("%Y-%m-%d") and int(bot_info.curent_date.strftime("%H")) == int((datetime.now()-timedelta(minutes=1)).strftime("%H")) or not any(bot_info.position_arr) and int(bot_info.curent_date.strftime("%H")) != int((datetime.now()-timedelta(minutes=1)).strftime("%H")):
        bot_info.curent_date = datetime.now()
        file = open(f'sandbox_results_{ticker}_{acc_id}.txt', 'a')
        file.write('=======================\n')
        print('=======================')
        hour = int(bot_info.curent_date.strftime("%H"))
        date_needed = bot_info.curent_date.strftime("%Y-%m-%d")
        bot_info.curent_date = pd.to_datetime(f'{date_needed} {hour}:00:00+00:00')
        print(bot_info.curent_date)
        file.write(f'{bot_info.curent_date}\n')
        bot_info.firstlevels = getlevels(bot_info.curent_date, ticker)
        bot_info.secondlevels = getlevels(bot_info.curent_date, ticker, first=False)
        print(bot_info.firstlevels, bot_info.secondlevels)
        file.write(f'{bot_info.firstlevels} {bot_info.secondlevels}\n')
        file.write(f'{bot_info.firstbuy}\n')
        file.write(f'{bot_info.secondbuy}')
        print('=======================')
        file.write('\n=======================\n')
        file.close()
    if not bot_info.position_arr[0]:
        if low_ <= bot_info.firstlevels[0]:
            file = open(f'sandbox_results_{ticker}_{acc_id}.txt', 'a')
            print('buy', datetime.now())
            file.write(f'buy {datetime.now()}\n')
            buy_1_response = buy(acc_id, ticker, bot_info.firstlevels[0], quantity)
            buy_1 = float(quotation_to_decimal(buy_1_response.initial_order_price))
            comission_1 = float(quotation_to_decimal(buy_1_response.initial_commission))
            bot_info.firstbuy = buy_1+comission_1
            bot_info.position_arr[0] = True
            file.close()
    elif bot_info.position_arr[0] and not bot_info.position_arr[1]:
        if low_ <= bot_info.secondlevels[0]:
            file = open(f'sandbox_results_{ticker}_{acc_id}.txt', 'a')
            print('buy second', datetime.now())
            file.write(f'buy second {datetime.now()}\n')
            buy_2_response = buy(acc_id, ticker, bot_info.secondlevels[0], quantity)
            buy_2 = float(quotation_to_decimal(buy_2_response.initial_order_price))
            comission_2 = float(quotation_to_decimal(buy_2_response.initial_commission))
            bot_info.secondbuy = buy_2+comission_2
            bot_info.position_arr[1] = True
            file.close()
        elif high_ >= bot_info.firstlevels[1]:
            file = open(f'sandbox_results_{ticker}_{acc_id}.txt', 'a')
            print('sell', datetime.now())
            file.write(f'sell {datetime.now()}\n')
            sell_1_responce = sell(acc_id, ticker, bot_info.firstlevels[1], quantity)
            bot_info.position_arr[0] = False
            sell_price = float(quotation_to_decimal(sell_1_responce.initial_order_price))
            comition = float(quotation_to_decimal(sell_1_responce.initial_commission))
            firstprofit = sell_price - comition - bot_info.firstbuy
            bot_info.profits += firstprofit
            file.close()
    elif bot_info.position_arr[1]:
        if high_ >= bot_info.secondlevels[1]:
            file = open(f'sandbox_results_{ticker}_{acc_id}.txt', 'a')
            print('sell second', datetime.now())
            file.write(f'sell second {datetime.now()}\n')
            sell_2_responce = sell(acc_id, ticker, bot_info.secondlevels[1], quantity)
            bot_info.position_arr[1] = False
            sell_price = float(quotation_to_decimal(sell_2_responce.initial_order_price))
            comition = float(quotation_to_decimal(sell_2_responce.initial_commission))
            secondprofit = sell_price - comition - bot_info.secondbuy
            bot_info.profits += secondprofit
            file.close()
    file_bot_info = open(f'sandbox_bot_info_{ticker}_{acc_id}.txt','w')
    file_bot_info.write(f'{bot_info.position_arr}\n{bot_info.profits}\n{bot_info.curent_date}\n{bot_info.firstlevels}\n{bot_info.secondlevels}\n{bot_info.firstbuy}\n{bot_info.secondbuy}')
    file_bot_info.close()

In [34]:
def request_iterator(figi):
    yield MarketDataRequest(
        subscribe_candles_request=SubscribeCandlesRequest(
            waiting_close=True,
            subscription_action=SubscriptionAction.SUBSCRIPTION_ACTION_SUBSCRIBE,
            instruments=[
                CandleInstrument(
                    figi=figi,
                    interval=SubscriptionInterval.SUBSCRIPTION_INTERVAL_ONE_MINUTE,
                )
            ],
        )
    )
    while True:
        time.sleep(1)

In [35]:
def start_sandbox_bot(acc_id, ticker, TOKEN, quantity):
    bot_info = make_bot_info(ticker, acc_id)
    print(bot_info.position_arr)
    print(bot_info.firstlevels)
    print(bot_info.secondlevels)
    figi = get_figi(ticker)
    with Client(TOKEN) as client:
        for marketdata in client.market_data_stream.market_data_stream(
            request_iterator(figi)
        ):
            if marketdata.candle!=None:
                high_ = marketdata.candle.high.units+marketdata.candle.high.nano*10**(-9)
                low_ = marketdata.candle.low.units+marketdata.candle.low.nano*10**(-9)
                new_strategy_logic(acc_id, ticker, low_, high_, bot_info, quantity)

### WORKING PART OF CODE

In [36]:
first_sandbox_acc_id = get_sandbox_account_id('contest2024:afanasevD/TinkContest:1')

In [ ]:
start_sandbox_bot(first_sandbox_acc_id, 'VTBR',TOKEN, 1)

[False, False]
(0.02, 0.02)
(0.02, 0.02)


C:\Users\Dmitrii\AppData\Roaming\Python\Python38\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


#### TRASH

In [ ]:
#insert_money(first_sandbox_acc_id, 8700)

In [ ]:
# def start_trading(Tickers, TOKEN, trading_sizes):
#     for i in range(len(Tickers)):
#         process = Process(target=start_sandbox_bot, args=(first_sandbox_acc_id,Tickers[i], TOKEN, trading_sizes[i],))
#         process.start()

In [ ]:
# Tickers = ['SBER','VTBR','RNFT','AFLT','TATN']
# trading_sizes = ['1','1','1','1','1']